In [1]:
import tensorflow as tf
import os
import random
from tensorflow.keras.layers import Rescaling

batch_size = 32

# splitting data
happy_paths = os.listdir('smiley_faces_dataset/happy')
sad_paths = os.listdir('smiley_faces_dataset/sad')

train_happy_paths = random.sample(happy_paths, int(len(happy_paths)*0.8))
test_happy_paths = [ p for p in happy_paths if p not in train_happy_paths]
# ensure no overlap:
overlap = [p for p in train_happy_paths if p in test_happy_paths]
print("len of happy overlap: ", len(overlap))


train_sad_paths = random.sample(sad_paths, int(len(sad_paths)*0.8))
test_sad_paths = [ p for p in sad_paths if p not in train_sad_paths]
# ensure no overlap:
overlap = [p for p in train_sad_paths if p in test_sad_paths]
print("len of sad overlap: ", len(overlap))

# ensure to copy the images to the directories
import shutil
for p in train_happy_paths:
    shutil.copyfile(os.path.join('smiley_faces_dataset/happy', p), os.path.join('smiley_faces_split/train/happy', p) )

for p in test_happy_paths:
    shutil.copyfile(os.path.join('smiley_faces_dataset/happy', p), os.path.join('smiley_faces_split/train/happy', p) )

for p in train_sad_paths:
    shutil.copyfile(os.path.join('smiley_faces_dataset/sad', p), os.path.join('smiley_faces_split/train/sad', p) )

for p in test_sad_paths:
    shutil.copyfile(os.path.join('smiley_faces_dataset/sad', p), os.path.join('smiley_faces_split/train/sad', p) )


2024-04-14 21:07:26.900863: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-14 21:07:27.023605: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-14 21:07:27.564865: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 21:07:27.564957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 21:07:27.661744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

len of happy overlap:  0
len of sad overlap:  0


In [2]:
img_height = 143
img_width = 107
batch_size = 32
train_data_dir = 'smiley_faces_split/train/'

# preprocessing
train_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="training",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="validation",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)


rescale = Rescaling(scale=1.0/255)
# Resize images to smaller dimensions'
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 216 files belonging to 2 classes.
Using 173 files for training.
Found 216 files belonging to 2 classes.
Using 43 files for validation.


In [3]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 1)),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D((2, 2)),
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(2, activation='softmax')
# ])

from keras import layers
from keras import models
import pandas as pd
from keras import optimizers


# Intializing a sequential model
model_cnn = models.Sequential()

# Adding first conv layer with 64 filters and kernel size 3x3 , padding 'same' provides the output size same as the input size
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(img_height, img_width, 3)))

# Adding max pooling to reduce the size of output of first conv layer
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# flattening the output of the conv layer after max pooling to make it ready for creating dense connections
model_cnn.add(layers.Flatten())

# Adding a fully connected dense layer with 100 neurons
model_cnn.add(layers.Dense(100, activation='relu'))

# Adding a fully connected dense layer with 84 neurons
model_cnn.add(layers.Dense(84, activation='relu'))

# Adding the output layer with * neurons and activation functions as softmax since this is a multi-class classification problem
model_cnn.add(layers.Dense(2, activation='softmax'))

# Compile model
# RMSprop (Root Mean Square Propagation) is commonly used in training deep neural networks.
# model_cnn.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
spec = train_rescale_ds.element_spec
spec

(TensorSpec(shape=(None, 143, 107, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [5]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 143, 107, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 72, 54, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 54, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 27, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 36, 27, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 18, 14, 32)        0

In [6]:
#fit the model from image generator
model_cnn.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
6/6 [==============================] - 220s 340ms/step - loss: 0.7137 - accuracy: 0.4509 - val_loss: 0.7009 - val_accuracy: 0.4419
Epoch 2/20
6/6 [==============================] - 2s 287ms/step - loss: 0.6992 - accuracy: 0.4855 - val_loss: 0.6866 - val_accuracy: 0.5581
Epoch 3/20
6/6 [==============================] - 2s 276ms/step - loss: 0.6903 - accuracy: 0.5145 - val_loss: 0.6918 - val_accuracy: 0.4884
Epoch 4/20
6/6 [==============================] - 2s 232ms/step - loss: 0.6827 - accuracy: 0.6185 - val_loss: 0.6994 - val_accuracy: 0.4884
Epoch 5/20
6/6 [==============================] - 2s 273ms/step - loss: 0.6843 - accuracy: 0.4913 - val_loss: 0.6877 - val_accuracy: 0.5349
Epoch 6/20
6/6 [==============================] - 2s 274ms/step - loss: 0.6449 - accuracy: 0.6416 - val_loss: 0.7362 - val_accuracy: 0.5581
Epoch 7/20
6/6 [==============================] - 2s 333ms/step - loss: 0.6733 - accuracy: 0.5780 - val_loss: 0.6765 - val_accuracy: 0.6279
Epoch 8/20
6/6 [==

In [8]:
model_cnn.save('models/cnn_first.keras')

In [9]:
tf.__version__

'2.15.0'